In [0]:
'''Create a logistic regression model in PySpark for mortgage portfolio and calculate Probability of Default (PD), Loss Given Default (LGD), Exposure at Default (EAD), and Expected Credit Loss (ECL)'''
'''
Steps:
1.Create a dataset for the mortgage portfolio.
2.Preprocess the data.
3.Build a Logistic Regression model using PySpark.
4.Calculate PD (Probability of Default) using the model.
5.Calculate LGD (Loss Given Default), EAD (Exposure at Default), and ECL (Expected Credit Loss).
Run the code on Databricks.'''


'''
Explanation of Key Concepts:
Logistic Regression Model:

We built a logistic regression model using various features like Credit_Score, Loan_Amount, Income, Interest_Rate, LTV, and Loan_Term to predict the likelihood of a customer defaulting on their mortgage (Default_Flag).
PD (Probability of Default):

The logistic regression model outputs a probability for each customer representing the likelihood of default (PD). A customer with a higher probability is more likely to default.
LGD (Loss Given Default):

Loss Given Default represents the portion of the loan that would be lost if the borrower defaults. It is assumed to be a function of the credit score. A low credit score leads to higher LGD.
EAD (Exposure at Default):

Exposure at Default represents the amount of loan at the time of default. In this simplified model, it is just the loan amount.
ECL (Expected Credit Loss):

ECL is calculated as PD * LGD * EAD. It is the total expected loss based on the probability of default, the loss given default, and the exposure at default.
'''

#Example Data:
#The dataset consists of 5 customers with the following columns:
#Customer_ID: Unique identifier for the customer.
#Credit_Score: Credit score of the customer.
#Loan_Amount: Amount of the mortgage loan.
#Income: Monthly income of the customer.
#Default_Flag: Whether the customer has defaulted (1 = default, 0 = no default).
#Interest_Rate: Interest rate on the mortgage.
#LTV: Loan-to-value ratio.
#Loan_Term: Duration of the loan in years.

# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, udf, when, lit
from pyspark.sql.types import DoubleType

# Step 1: Initialize Spark Session
spark = SparkSession.builder.appName("MortgageDefaultPrediction").getOrCreate()

# Step 2: Example dataset for mortgage portfolio (simplified dataset)
data = [
    (1, 750, 300000, 5000, 1, 0.03, 0.7, 0.9),  # Customer 1: Good credit, paid well, etc.
    (2, 650, 150000, 3000, 0, 0.05, 0.6, 0.8),  # Customer 2: Average credit, defaulted
    (3, 680, 250000, 4000, 1, 0.04, 0.65, 0.85), # Customer 3: Good credit, paid well, etc.
    (4, 620, 100000, 2000, 0, 0.1, 0.5, 0.7),   # Customer 4: Poor credit, defaulted
    (5, 700, 400000, 6000, 1, 0.02, 0.75, 0.95)  # Customer 5: Good credit, paid well
]

# Define the schema for the mortgage dataset
columns = ['Customer_ID', 'Credit_Score', 'Loan_Amount', 'Income', 'Default_Flag', 'Interest_Rate', 'LTV', 'Loan_Term']

# Create a DataFrame from the data
df = spark.createDataFrame(data, columns)

# Show the dataset
df.show()

# Step 3: Data Preprocessing: Feature Engineering
# We will create a feature vector using Credit_Score, Loan_Amount, Income, Interest_Rate, LTV, Loan_Term
assembler = VectorAssembler(inputCols=['Credit_Score', 'Loan_Amount', 'Income', 'Interest_Rate', 'LTV', 'Loan_Term'],
                            outputCol='features')

# Step 4: Splitting the data into training and test sets (80% training, 20% testing)
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Step 5: Create a Logistic Regression model
lr = LogisticRegression(labelCol='Default_Flag', featuresCol='features')

# Create a pipeline
pipeline = Pipeline(stages=[assembler, lr])

# Train the model
model = pipeline.fit(train_data)

# Step 6: Make predictions on the test data
predictions = model.transform(test_data)

# Show predictions
predictions.select('Customer_ID', 'prediction', 'probability').show()

# Step 7: Extract the Probability of Default (PD) from the 'probability' column
# The 'probability' column is a STRUCT, and 'values' contains the probabilities.
predictions = predictions.withColumn('PD', col('probability').getItem('values').getItem(1))

# Show PD for each customer
predictions.select('Customer_ID', 'PD').show()

# Step 8: Calculate LGD (Loss Given Default) based on Credit_Score
# Assuming a fixed LGD rate based on the credit score.
def calculate_LGD(credit_score):
    if credit_score < 650:
        return 0.9  # High loss for low credit score
    elif credit_score < 700:
        return 0.7  # Moderate loss for average credit score
    else:
        return 0.5  # Lower loss for good credit score

# Register UDF to calculate LGD
lgd_udf = udf(calculate_LGD, DoubleType())

# Add LGD column based on Credit_Score (with null handling)
predictions = predictions.withColumn('LGD', 
                                     when(col('Credit_Score').isNotNull(), lgd_udf(col('Credit_Score')))
                                     .otherwise(lit(0)))  # Default to 0 if Credit_Score is null

# Step 9: Add a column for EAD (Exposure at Default) as Loan_Amount
predictions = predictions.withColumn('EAD', col('Loan_Amount'))

# Step 10: Calculate ECL (Expected Credit Loss)
predictions = predictions.withColumn('ECL', col('PD') * col('LGD') * col('EAD'))

# Step 11: Show the final results with PD, LGD, EAD, and ECL
predictions.select('Customer_ID', 'PD', 'LGD', 'EAD', 'ECL').show()

# Stop the Spark session
spark.stop()


+-----------+------------+-----------+------+------------+-------------+----+---------+
|Customer_ID|Credit_Score|Loan_Amount|Income|Default_Flag|Interest_Rate| LTV|Loan_Term|
+-----------+------------+-----------+------+------------+-------------+----+---------+
|          1|         750|     300000|  5000|           1|         0.03| 0.7|      0.9|
|          2|         650|     150000|  3000|           0|         0.05| 0.6|      0.8|
|          3|         680|     250000|  4000|           1|         0.04|0.65|     0.85|
|          4|         620|     100000|  2000|           0|          0.1| 0.5|      0.7|
|          5|         700|     400000|  6000|           1|         0.02|0.75|     0.95|
+-----------+------------+-----------+------+------------+-------------+----+---------+

+-----------+----------+--------------------+
|Customer_ID|prediction|         probability|
+-----------+----------+--------------------+
|          1|       1.0|[2.33146222098297...|
+-----------+----------

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4214444101529135>:96
     92 predictions.select('Customer_ID', 'prediction', 'probability').show()
     94 # Step 7: Extract the Probability of Default (PD) from the 'probability' column
     95 # The 'probability' column is a STRUCT, and 'values' contains the probabilities.
---> 96 predictions = predictions.withColumn('PD', col('probability').getItem('values').getItem(1))
     98 # Show PD for each customer
     99 predictions.select('Customer_ID', 'PD').show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     retur